In [1]:
#!/usr/bin/python 
# -*- coding: utf-8 -*-
!pip install bs4==0.0.1
!pip install elasticsearch==7.10.0

In [2]:
def random_sleep():
    from random import randint
    from time import sleep
    sleep_time = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]
    index = randint(0, len(sleep_time)-1)
    sleep(sleep_time[index])

In [3]:
def random_header():
    from random import randint
    header_list = [
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/14.0.835.163 Safari/535.1",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:6.0) Gecko/20100101 Firefox/6.0",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
    "Opera/9.80 (Windows NT 6.1; U; zh-cn) Presto/2.9.168 Version/11.50",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 2.0.50727; SLCC2; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; InfoPath.3; .NET4.0C; Tablet PC 2.0; .NET4.0E)",
    "Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; InfoPath.3)",
    "Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; GTB7.0)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1)",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; ) AppleWebKit/534.12 (KHTML, like Gecko) Maxthon/3.0 Safari/534.12",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; InfoPath.3; .NET4.0C; .NET4.0E)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; InfoPath.3; .NET4.0C; .NET4.0E; SE 2.X MetaSr 1.0)",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.3 (KHTML, like Gecko) Chrome/6.0.472.33 Safari/534.3 SE 2.X MetaSr 1.0",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; InfoPath.3; .NET4.0C; .NET4.0E)",
    "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.41 Safari/535.1 QQBrowser/6.9.11079.201",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; InfoPath.3; .NET4.0C; .NET4.0E) QQBrowser/6.9.11079.201",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)"]
    index = randint(0, len(header_list)-1)
    return {"user-agent":header_list[index]}

In [4]:
def re_filter(content):
    '''
    使用re工具, 处理冗余信息
    '''
    import re

    pattern1 = re.compile(r'#[^#]+#')
    pattern2 = re.compile(r"[a-zA-Z]")
    return re.sub(pattern2, "", re.sub(pattern1, "", content))

In [29]:
def get_info(res, title_name="", get_content=True):
    '''
    处理json数据, 获取指定的内容(题目和评论)或其他内容
    '''
    from bs4 import BeautifulSoup
    from time import sleep

    if res.status_code != 200:
        print("ip被封!, 等候20s")
        sleep(20)
        if get_content:
            return -1, 0
        else:
            return []
    print(res.status_code)

    contents_dict = res.json()["data"]
    if get_content:
        # 判断是否为浏览页
        ## 如果是
        if "max_id" not in contents_dict:
            mid_list = []
            for item in contents_dict["cards"]:
                # 如果mid_list大于2, 就跳出
                if len(mid_list)>=2:
                    break
                # 判断评论类型
                if item["card_type"] != 9:
                    continue
                # 获取mid
                mid_list.append(item["mblog"]["id"])
            return 1, mid_list
        else:  
        # 如果不是
            contents_list = []
            # 获取下一页
            max_id = contents_dict["max_id"]
            print(max_id)
            for item in contents_dict["data"]:

                # 获取评论内容和时间
                content = item["text"]
                # created_time = item["created_at"]

                # 使用re工具, 将##内容全部清除
                content = re_filter(content)
                contents_list.append((title_name, content))
            return max_id, contents_list
    else:
        keyword_list = []
        for item in contents_dict["cards"][0]["group"]:
            content = ""
            keyword = str(item["title_sub"])
            keyword_list.append(keyword)
        return keyword_list

In [6]:
def push_es(content_list):
    '''
    将数据推入云es库中
    '''
    # 考虑列表为空时, 直接跳出
    if len(content_list)==0:
        return
    
    # 列表非空时, 开始推入
    from elasticsearch import Elasticsearch
    from elasticsearch import helpers
    # 开始连接es库(感觉这里可以优化)
    es = Elasticsearch(
        ['es-cn-09k1xdlqf000a7yyn.public.elasticsearch.aliyuncs.com'],
        http_auth=('elastic', 'Zy13720028637'),
        port=9200,
        use_ssl=False
    )
    # 判断数据是否为字典类型
    if type(content_list[0]) != dict:
        names = ["title", "describe"]
        for i, data in enumerate(content_list):
            temp_dict = {}
            for index, content in enumerate(data):
                temp_dict[names[index]] = content
            content_list[i] = temp_dict
    # 将数据修饰一下
    action = ({
        "_index":"zhihu",
        "_type":"question",
        "_source":data
    } for data in content_list)
    # 导入数据
    helpers.bulk(es, action)

In [20]:
def web_grap(keyword):
    '''
    网页抓取
    '''
    import requests
    import pandas as pd
    
    base_url = "https://m.weibo.cn/api/container/getIndex?containerid=100103type%3D1%26q%3D{}&page_type=searchall&page={}"
    # headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 Edg/86.0.622.63"}
    print("\n抓取热度:{}".format(keyword))

    # 第一步抓取热点公众号
    while True:
        url = base_url.format(keyword, 0)
        random_sleep()
        res = requests.get(url.format(keyword, 1), headers=random_header())
        status, mid_list = get_info(res)
        # 判断ip是否被封
        if status==-1:
            print("再次申请...")
            continue
        else:
            break
    
    base_url = "https://m.weibo.cn/comments/hotflow?id={}&mid={}&max_id_type=0&max_id={}"
    max_id = 0
    num = 0
    for i in range(len(mid_list)):
        while True:
            # 正常抓取
            url = base_url.format(mid_list[i], mid_list[i], max_id)
            print(url)
            random_sleep()
            res = requests.get(url, headers=random_header())
            max_id, contents_list = get_info(res, keyword)

            # 判断ip是否被封
            if max_id==-1:
                print("  再次申请...")
                continue
            num += len(contents_list)

            ## 存储数据到es中
            push_es(contents_list)
            print(" 已抓取{}条评论!!".format(num))

            # 判断是否满足跳出条件
            if max_id == 0:
                print("{}抓取结束!".format(keyword))
                break


In [8]:
def to_more_grap():
    '''
    用于获取相关热度的关键词
    '''
    import requests
    
    # 准备网站
    base_url = "https://m.weibo.cn/api/container/getIndex?containerid=231583&page_type=searchall"
    
    ## 准备https头部
    # headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 Edg/86.0.622.63"}
    # ## 睡眠
    random_sleep()
    ## 开始访问
    res = requests.get(base_url, headers=random_header())
    ## 处理json数据, 获取更多问题号
    contents_list = get_info(res, get_content=False)
    return contents_list

In [9]:
def set_grap(question_num=1):
    '''
    集成爬虫
    '''
    # 爬取关键词
    keyword_list = to_more_grap()
    web_grap(keyword_list[0])
    question_num -= 1

    i = 1
    while question_num:
        temp_i = i
        while len(keyword_list) == i:
            keyword_list += to_more_grap()
            keyword_list = sorted(set(keyword_list),key=keyword_list.index)
            temp_i -= 1
            if temp_i == -1:
                print("因热度关键词一致, 故抓取结束!")
                return
        web_grap(keyword_list[i])
        i += 1
        question_num -= 1
    print("抓取任务结束!")

In [10]:
def weibo_grap(num=5):
    set_grap(num)

In [11]:
def es_search(search_in):
    '''
    根据search_in(指定字符串)查找相关问题, 并返回全部内容
    '''
    from elasticsearch import Elasticsearch
    # 开始连接es库(感觉这里可以优化)
    es = Elasticsearch(
        ['es-cn-09k1xdlqf000a7yyn.public.elasticsearch.aliyuncs.com'],
        http_auth=('elastic', 'Zy13720028637'),
        port=9200,
        use_ssl=False
    )
    
    # 开始查询(这里绝对可以优化, 先简单实现)
    dsl = {
        'query': {
            'match': {
                'title': search_in
            }
        }
    }

    return es.search(index='zhihu', doc_type='question', body=dsl)

In [12]:
def get_search_info(content):
    '''
    先简单一点, 就获取相关问题title
    '''
    title_list = []
    for data in content["hits"]["hits"]:
        if len(title_list)>=5:
            break
        source = data["_source"]
        title = source["title"]
        if title not in title_list:
            title_list.append(title)
    return title_list

In [13]:
def info_print(content_info):
    '''
    简单实现, 就打印含有title的列表
    '''
    print("  搜索结果:")
    for index, title in enumerate(content_info):
        print("    {}. {}".format(index+1, title))

In [14]:
def weibo_search():
    # 输入相关查询
    search_in = input()
    # 从es库查询结果
    content = es_search(search_in)
    # 处理信息, 获取主要内容
    content_info = get_search_info(content)
    # 打印内容
    info_print(content_info)

In [28]:
if __name__=="__main__":
    while True:
        print("------执行菜单------")
        print("1. 简单抓取")
        print("2. 相关搜索")
        print("3. 退出程序")
        select_num = int(input())
        if select_num==1:
            weibo_grap(1)
        elif select_num==2:
            weibo_search()
        else:
            print("成功退出!")
            break

------执行菜单------
1. 简单抓取
2. 相关搜索
3. 退出程序
{"ok":1,"data":{"hotwords":[{"word":"\u4e2d\u56fd\u6570\u636e\u5e9340\u5e74\u9996\u6b21\u8fdb\u5165\u5168\u7403\u9876\u7ea7\u884c\u5217","note":"\u4e2d\u56fd\u6570\u636e\u5e9340\u5e74\u9996\u6b21\u8fdb\u5165\u5168\u7403\u9876\u7ea7\u884c\u5217","tip":"\u5927\u5bb6\u6b63\u5728\u641c\uff1a"}],"cardlistInfo":{"v_p":"42","containerid":"231583","title_top":"","total":0,"show_style":1,"starttime":1606306014,"can_shared":0,"page_size":"10","volume":0,"cardlist_menus":[],"page":null},"cards":[{"col":2,"itemid":"hot_search","card_type":17,"group":[{"action_log":{"act_code":554,"ext":"t:30|pos:1_0|c_type:30|cate_type:hotword|hot_word:#\u6768\u8d85\u8d8a\u843d\u6237\u4e0a\u6d77#|cate:48|version:1|page:1|type:1","luicode":"","fid":"231583","lfid":"","uicode":""},"icon":"https:\/\/simg.s.weibo.com\/20200928_fei_small.png","scheme":"https:\/\/m.weibo.cn\/search?containerid=100103type%3D1%26t%3D10%26q%3D%23%E6%9D%A8%E8%B6%85%E8%B6%8A%E8%90%BD%E6%88%B7%E4%B8%8A

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
1